In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib widget
import copy
import numpy as np
 
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import rc, rcParams

home_folder = './'

from tqdm.notebook import tqdm
from scipy.integrate import quad

import pandas as pd
import networkx as nx

np.random.seed(0)

from helpers import *
from framework import *
from selection_and_validation_framework import *

## Network simulation

In [ ]:
beta = 0.5
n0 = 50
n = 10000 - n0

G = nx.erdos_renyi_graph(n0, p=0.1, seed=None, directed=False)

rand_bits = np.random.random(size=n0) > 0.5
nx.set_node_attributes(G, {i: rand_bits[i] for i in range(n0)}, name='gender')
# gender 1 is male and 0 is female


rand_bits = np.random.random(size=n) > 0.5
rand_nums = np.random.random(size=n) * 100


for i in tqdm(range(n)):
    ind = i + n0

    pdf = np.array([G.degree[j] * (beta + (1-beta) * G.nodes[j]['gender']) for j in range(i + n0)])
    pdf /= sum(pdf)

    conns = np.random.choice(size=1, a=list(G.nodes), p=pdf, replace=False)

    G.add_node(ind)
    G.nodes[ind]['gender'] = rand_bits[i]
    for conn in conns:
        G.add_edge(conn, ind)

In [ ]:
male_degree_dist = []
female_degree_dist = []

for v in list(G.nodes):
    if G.nodes[v]['gender'] == 1:
        male_degree_dist.append(G.degree[v])
    else:
        female_degree_dist.append(G.degree[v])

In [ ]:
plt.figure().clear()

plt.hist(female_degree_dist, bins=200, label='female degrees', alpha=0.5, density=True)

plt.hist(male_degree_dist, bins=200, label='male degrees', alpha=0.5, density=True)


plt.legend(fontsize=22)
plt.xlim(0,500)


plt.show() 

In [ ]:
print(f'mean of latent={np.mean(male_degree_dist)} vs. mean of biased={np.mean(female_degree_dist)}') 

In [ ]:
male_degree_dist_scaled = np.array(male_degree_dist) / 50
female_degree_dist_scaled = np.array(female_degree_dist) / 50

Omega = get_omega([male_degree_dist_scaled, female_degree_dist_scaled])
pdf_latent = get_pdf_from_utils(male_degree_dist_scaled, Omega)
pdf_biased = get_pdf_from_utils(female_degree_dist_scaled, Omega)
f_0 = get_pdf_uniform_omega(Omega)

kl_div_fd = 0 * ONE
for i, p in enumerate(pdf_latent):
    if p > 0:
        if USE_LONG_MATH:
            kl_div_fd += p * mp.log(p / f_0(Omega[i]))
        else:
            kl_div_fd += p * np.log(p / f_0(Omega[i]))
print(f'kl_div_fd = {kl_div_fd}')

In [ ]:
print(f'original tv distance: {sum(abs(pdf_latent * ONE - pdf_biased)) / 2}')

### Validation

In [ ]:
alpha_list = np.logspace(-4, 2, 100)
tau_list = np.logspace(0.1, 1, 10)

prior_list = [get_pdf_uniform_omega(omega=Omega)]
names_of_prior = ['uniform']

In [ ]:
generate_plots(latent_utilities = male_degree_dist_scaled, biased_utilities = female_degree_dist_scaled, 
                err_func=err_func_pareto, alpha_list=alpha_list, tau_list=tau_list, plot_fd=False,
                prior_list=prior_list, names_of_prior=names_of_prior, print_verbose=False, plot_prior=False, plot=False, xlim = (0,0.3), FAIL_CNT_THRESH=1000)

In [ ]:
generate_plots(latent_utilities = male_degree_dist_scaled, biased_utilities = female_degree_dist_scaled, 
                err_func=err_func_pareto, alpha_list=alpha_list, tau_list=tau_list, plot_fd=False,
                prior_list=prior_list, names_of_prior=names_of_prior, print_verbose=False, plot_prior=False, plot=False, xlim = (0,0.3), FAIL_CNT_THRESH=1000)

In [ ]:
generate_plots(latent_utilities = male_degree_dist_scaled, biased_utilities = female_degree_dist_scaled, 
                err_func=err_func_pareto, alpha_list=alpha_list, tau_list=tau_list, plot_fd=False,
                prior_list=prior_list, names_of_prior=names_of_prior, print_verbose=False, plot_prior=False, plot=False, xlim = (0,0.3), FAIL_CNT_THRESH=1000)

#### Multiplicative bias

In [ ]:
def ok(i, tmp): return i >= 0 and i < len(tmp)

best_tv = 1e10
mp = {v: i      for i, v in enumerate(Omega)}
best_param = {'beta': -1, 'shift': 0}

Omega = get_omega([male_degree_dist_scaled, female_degree_dist_scaled])
pdf_latent = get_pdf_from_utils(male_degree_dist_scaled, omega=Omega)
pdf_biased = get_pdf_from_utils(female_degree_dist_scaled, omega=Omega)

mu = 0
pbar = tqdm([0] + list(range(-len(Omega), len(Omega), 1)))
for shift in pbar: 
    for beta in np.logspace(-2, 0, 1000):
        pdf_syn_biased_utilities_ = np.zeros_like(pdf_latent)

        for i, v in enumerate(Omega):
            w = beta * (v + min(Omega)) - min(Omega)
            w = Omega[   np.searchsorted(Omega, w)   ]
            pdf_syn_biased_utilities_[mp[w]] += pdf_latent[i] 
             
        cur_tv = sum(abs(pdf_syn_biased_utilities_/sum(pdf_syn_biased_utilities_) - pdf_biased/sum(pdf_biased))) / 2

        if cur_tv < best_tv:
            best_tv = cur_tv
            best_param['beta'] = beta
            best_param['shift'] = shift

plt.figure().clear()
x = list(Omega)
print(f'best_tv={best_tv}')
beta = best_param['beta']
print(f'beta={beta}')

pdf_syn_biased_utilities_ = np.zeros_like(pdf_latent)

for i, v in enumerate(Omega):
    w = beta * (v + min(Omega)) - min(Omega)
    w = Omega[   np.searchsorted(Omega, w)   ]
    pdf_syn_biased_utilities_[mp[w]] += pdf_latent[i]
 
plt.plot(x, pdf_syn_biased_utilities_, label='Synthetic Biased Utilities', linewidth=4)
plt.plot(x, pdf_biased, label='Biased Utilities in Data', linewidth=4)

plt.legend()
plt.show()
plt.close()

#### Implicit variance  

In [ ]:
def add_gaussian(pdf_latent, omega, mu, sigma):
    pdf_norm = lambda x: np.exp(  - (x-mu)**2 / 2 / sigma / sigma  ) /  np.sqrt( 2 * np.pi) / sigma

    biased_utilities = np.zeros(len(omega))
    for i, v in enumerate(omega):
        for j, w in enumerate(omega):
            biased_utilities[i] += pdf_norm(v - w) * pdf_latent[j]

    return biased_utilities / sum(biased_utilities)

In [ ]:
best_tv = 1e10
best_param = {'mu': 0, 'sigma': -1, 'shift': 0}

Omega = get_omega([male_degree_dist_scaled, female_degree_dist_scaled])
pdf_latent = get_pdf_from_utils(male_degree_dist_scaled, omega=Omega)
pdf_biased = get_pdf_from_utils(female_degree_dist_scaled, omega=Omega)
 
mu = 0
pbar = tqdm([0] + list(range(-len(Omega), len(Omega), 1)))
for shift in pbar:
    pbar.set_description(f"best_tv={np.round(best_tv, 2)} || best_param={np.round(best_param['mu'], 2), np.round(best_param['sigma'], 2), best_param['shift']}")
    for sigma in np.logspace(-2, 1, 1000):
        pdf_syn_biased_utilities_ = add_gaussian(pdf_latent=pdf_latent, omega=Omega, mu=mu, sigma=sigma)
        tmp = copy.deepcopy(pdf_syn_biased_utilities_)

        for i in range(len(pdf_syn_biased_utilities_)):
            if i + shift >= 0 and i + shift < len(tmp):
                pdf_syn_biased_utilities_[i] = tmp[i + shift]
            else:
                pdf_syn_biased_utilities_[i] = 0

        cur_tv = sum(abs(pdf_syn_biased_utilities_ - pdf_biased)) / 2

        if cur_tv < best_tv:
            best_tv = cur_tv
            best_param['mu'] = mu
            best_param['sigma'] = sigma 
            best_param['shift'] = shift

plt.figure().clear()
x = list(Omega)
print(f'best_tv={best_tv}')

pdf_syn_biased_utilities_ = add_gaussian(pdf_latent=pdf_latent, omega=Omega, mu=best_param['mu'], sigma=best_param['sigma'])
shift = best_param['shift']
tmp = copy.deepcopy(pdf_syn_biased_utilities_)
for i in range(len(pdf_syn_biased_utilities_)):
    if i + shift >= 0 and i + shift < len(tmp):
        pdf_syn_biased_utilities_[i] = tmp[i + shift]
    else:
        pdf_syn_biased_utilities_[i] = 0
                
plt.plot(x, pdf_syn_biased_utilities_, label='Synthetic Biased Utilities', linewidth=4)
plt.plot(x, pdf_biased, label='Biased Utilities in Data', linewidth=4)

plt.legend()
plt.show()
plt.close()